In [1]:
import pandas as pd
import numpy as np

In [2]:
from os import listdir
from os.path import isfile, join

path = "../price data/iDhP/"
files = [f for f in listdir(path) if isfile(join(path, f))]

In [3]:
files

['candles_bch-usd_1h.csv',
 'candles_eos-usd_1h.csv',
 'candles_neo-usd_1h.csv',
 'candles_etc-usd_1h.csv',
 'candles_btc-usd_1h.csv',
 'candles_link-usd_1h.csv',
 'candles_eth-usd_1h.csv',
 'candles_ada-usd_1h.csv',
 'candles_trx-usd_1h.csv',
 'candles_atom-usd_1h.csv',
 'candles_xmr-usd_1h.csv',
 'candles_zec-usd_1h.csv',
 'candles_xlm-usd_1h.csv',
 'candles_xtz-usd_1h.csv',
 'candles_xrp-usd_1h.csv',
 'candles_ltc-usd_1h.csv']

In [7]:
volume_col = 'volume_quote'

feature_names = [volume_col]

for file in files:
    data = pd.DataFrame()

    #read data
    file_path = path+file
    d = pd.read_csv(file_path)
    d['timestamp'] = [pd.Timestamp(t) for t in d['timestamp']]
    
    #daily average
    d['day'] = [ts.strftime('%Y-%m-%d') for ts in d['timestamp'].to_list()]
    d = d.groupby(by=['day']).mean()
    d.index = [pd.Timestamp(ts) for ts in d.index.to_list()]
    d['day'] = d.index
    d['market'] = file[8:15]
    
    x = d[['market','day',volume_col]].copy()
    
    #generate sma of price
    #x[feature_names[1]] = x.rolling(window=int(feature_names[1][-2:]))['close'].mean()
    #x[feature_names[2]] = x.rolling(window=int(feature_names[2][-2:]))['close'].mean()
    
    for feature in feature_names:
        x['log_'+feature] = np.log(x[feature])
        x['diff_'+feature] = x[feature].diff()
        x['log_diff_'+feature] = x['log_'+feature].diff()
        x['diff_ratio_'+feature] = (x['diff_'+feature]/x[feature])*100
    
    data = pd.concat([data,x],ignore_index=True)
    data.to_csv("generated_volume_data/"+file[8:15]+"_daily.csv")

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/algorithms.py:1926: RuntimeWarning: invalid value encountered in subtract
  out_arr[res_indexer] = arr[res_indexer] - arr[lag_indexer]
